## Descriptive analyses for MOOC item response matrices

In [1]:
import os

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Retrieve list of courses.

In [3]:
home = os.path.expanduser('~')
data_dir = home + "/Code/irt/data/"

courses = []
with open(data_dir + "courselist.txt") as c_list:
    for course in c_list:
        courses.append(course.replace('/', '_').rstrip())

print courses

['Medicine_MedStats_Summer2014', 'HumanitiesandScience_StatLearning_Winter2015']


In [ ]:
measures = ['first_view', 'first_attempt', 'last_attempt', 'time_to_first_attempt', 'time_to_last_attempt',
            'time_spent_attempting', 'n_attempts', 'first_grade', 'last_grade']

